In [ ]:
#Group Information:

#Name                   | Roll Number
#---------------------------------------
#Aditya Aman            | NS25Z035
#Pravitt Sethi          | NS25Z022
#Srivatsan Sarvesan     | DA24E001

In [2]:
import random
import numpy as np
from math import radians, cos, sin, asin, sqrt
from collections import defaultdict, deque
import networkx as nx
import json

The implementation follows these steps:

1.Create router nodes from the provided information

2.Calculate distances between all pairs of routers using the Haversine formula

3.Randomly remove some connections based on a threshold parameter (T=0.3)

4.Establish the remaining connections as the network topology

This approach realistically models a network where not all potential connections exist. The threshold value of 0.3 was carefully chosen as a balance - a smaller value would create too few connections, while a larger value would create an unrealistically dense network.

In [3]:
# Haversine distance function (from your existing code)
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Earth radius in km
    return c * r

Loading the routers info

In [4]:
with open('router.json', 'r') as file:
    routers = json.load(file)

Converting it into list for easy operation 

In [5]:
router_list = [
    (ip, float(info['Latitude']), float(info['Longitude']))
    for ip, info in routers.items()
]

Marking all the routers as nodes and dividing them into clusters where each cluster is basically those router which are located less than 50km from each other

In [6]:
G = nx.Graph()

# Add all routers as nodes
for ip, lat, lon in router_list:
    G.add_node(ip, lat=lat, lon=lon)

# Connect routers within 50km
for i, (ip1, lat1, lon1) in enumerate(router_list):
    for j, (ip2, lat2, lon2) in enumerate(router_list[i+1:], i+1):
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 50:
            G.add_edge(ip1, ip2)

In [7]:
clusters = list(nx.connected_components(G))

Making common latitude and longitutde for each clusters and marking them from 1 to N

In [8]:
# Create a lookup dictionary from IP to (lat, lon)
ip_to_coords = {ip: (lat, lon) for ip, lat, lon in router_list}

cluster_coordinates = []

for cluster_id, cluster in enumerate(clusters):
    latitudes = [ip_to_coords[ip][0] for ip in cluster]
    longitudes = [ip_to_coords[ip][1] for ip in cluster]
    cluster_coordinates.append([
        cluster_id,
        max(latitudes),
        max(longitudes)
    ])

print(cluster_coordinates)


[[0, 13.0878, 80.2785], [1, 19.0728, 72.8826], [2, 1.2897, 103.8501], [3, -8.8368, 13.2343], [4, 43.297, 5.3811], [5, 52.374, 4.8897], [6, -26.2023, 28.0436], [7, 45.4643, 9.1895], [8, 38.7167, -9.1333], [9, 12.3657, -1.5339], [10, 6.4541, 3.3947], [11, 43.2627, -2.9253], [12, 41.5503, -8.42], [13, 4.0483, 9.7043], [14, -33.9258, 18.6167], [15, 34.0522, -118.2437], [16, 32.7831, -96.8067], [17, 38.1166, 13.3636], [18, 32.8874, 13.1873], [19, 51.5085, -0.1257], [20, 48.8534, 2.3488], [21, -18.9137, 47.5361], [22, -18.1492, 49.4023], [23, 12.6091, -7.9752], [24, 40.4165, -3.7026], [25, -29.1211, 26.214], [26, -4.0547, 39.6636], [27, -1.2833, 36.8167], [28, 0.3163, 32.5822], [29, -1.95, 30.0588], [30, 14.6937, -17.4441], [31, -12.8024, 28.2132], [32, -15.4067, 28.2871]]


In [9]:
len(cluster_coordinates)

33

Defining the  class Router Node as defined in Ex-1

In [10]:
class RouterNode:
    def __init__(self, router_id, ip, latitude, longitude):
        self.id = router_id
        self.ip = ip
        self.lat = latitude
        self.lon = longitude
        self.neighbors = {}  # neighbor_id: weight (distance)

Defining the threshold

In [11]:
T=0.3 

# The reason for taking threshold as 0.3 as 0.1 would have been too small and 0.5 would have been too large.

 Build graph from routers list (routers_info: list of (ip, lat, lon))

In [12]:
def build_graph(routers_info, threshold=T):
    n = len(routers_info)
    routers = {}
    # Create RouterNode objects with IDs
    for i, (ip, lat, lon) in enumerate(routers_info, start=1):
        routers[i] = RouterNode(i, ip, float(lat), float(lon))
    
    # Create complete graph edges with haversine distance
    edges = []
    for i in range(1, n+1):
        for j in range(i+1, n+1):
            dist = haversine(routers[i].lon, routers[i].lat, routers[j].lon, routers[j].lat)
            edges.append((i, j, dist))
    
    # Randomly drop T fraction of edges
    num_edges_to_drop = int(threshold * len(edges))
    edges_to_drop = set(random.sample(range(len(edges)), num_edges_to_drop))
    
    # Add edges to routers if not dropped
    for idx, (u, v, w) in enumerate(edges):
        if idx in edges_to_drop:
            continue
        routers[u].neighbors[v] = w
        routers[v].neighbors[u] = w
    
    return routers

Generating graphs using the previously created clusters

In [13]:
routers=build_graph(cluster_coordinates, threshold=T)

In [14]:
routers[1].neighbors

{4: 7793.3109808335175,
 7: 7138.685366393103,
 9: 9051.908804922785,
 10: 8828.624190250373,
 11: 8430.035806905746,
 12: 8469.141860672988,
 13: 8940.181875595046,
 14: 7803.264562216937,
 15: 8349.944357036766,
 16: 14418.993931504492,
 18: 7103.165030074891,
 19: 7095.362750218246,
 20: 8210.807545251884,
 22: 5052.944317396905,
 23: 4851.009226594174,
 24: 9507.551485856951,
 25: 8564.491478094622,
 27: 4872.53473982718,
 28: 5052.0311808854785,
 29: 5442.1493775747285,
 30: 5786.199154367221,
 31: 10448.565471051885,
 33: 6537.929261964411}

# (a)Link State Routing Protocol Implementation

In [15]:

def link_state_routing(routers):
    n = len(routers)
    # Each router floods its LSA to all others
    # Initially each router knows only its neighbors
    # LSDB: router_id -> dict of neighbors and weights
    LSDB = {r: dict(routers[r].neighbors) for r in routers}
    
    # Flooding rounds: in each round, routers share LSAs they know
    # We'll simulate flooding until all routers have full LSDB
    rounds = 0
    converged = False
    
    # Each router's LSDB copy
    router_LSDBs = {r: {r: LSDB[r]} for r in routers}
    
    while not converged:
        rounds += 1
        updated = False
        # Each router sends its LSDB to neighbors
        for r in routers:
            for nbr in routers[r].neighbors:
                # Merge LSDBs
                before = len(router_LSDBs[nbr])
                router_LSDBs[nbr].update(router_LSDBs[r])
                after = len(router_LSDBs[nbr])
                if after > before:
                    updated = True
        if not updated:
            converged = True
    
    # Total LSAs flooded = number of routers * rounds
    total_LSAs = n * rounds
    
    # Number of entries in LSDB = number of routers
    entries_in_LSDB = n
    
    return total_LSAs, entries_in_LSDB, rounds, router_LSDBs

Implementing the link state routing protocol on the routers

In [16]:
total_LSAs, entries_in_LSDB, rounds, router_LSDBs=link_state_routing(routers)

In [17]:
print("Total LSAs flooded:", total_LSAs)
print("Entries in LSDB:", entries_in_LSDB)
print("Rounds until convergence:", rounds)

Total LSAs flooded: 99
Entries in LSDB: 33
Rounds until convergence: 3


This function simulates the flooding process where:

Each router starts with knowledge only of its direct neighbors

In each round, routers share their Link State Database (LSDB) with neighbors

The process continues until no new information is learned (convergence)

The function tracks important metrics like total LSAs flooded and rounds until convergence

<b>The output shows that the network converged in 3 rounds with 99 total LSAs flooded, demonstrating the efficiency of the protocol.

# (b) Function that takes router ID and prints the forwarding table

-> Computing the forwarding table

In [18]:
def compute_forwarding_tables(routers, router_LSDBs=router_LSDBs):
    forwarding_tables = {}
    
    for r in routers:
        # Build graph from LSDB of router r
        graph = defaultdict(dict)
        for node, nbrs in router_LSDBs[r].items():
            for nbr, w in nbrs.items():
                graph[node][nbr] = w
        # Dijkstra from r
        dist = {node: float('inf') for node in graph}
        prev = {node: None for node in graph}
        dist[r] = 0
        unvisited = set(graph.keys())
        
        while unvisited:
            u = min(unvisited, key=lambda x: dist[x])
            unvisited.remove(u)
            for v, w in graph[u].items():
                alt = dist[u] + w
                if alt < dist[v]:
                    dist[v] = alt
                    prev[v] = u
        
        # Build forwarding table: destination -> next hop
        forwarding = {}
        for dest in graph:
            if dest == r:
                continue
            # Trace path back to find next hop
            next_hop = dest
            while prev[next_hop] != r and prev[next_hop] is not None:
                next_hop = prev[next_hop]
            forwarding[dest] = next_hop if prev[next_hop] is not None else None
        forwarding_tables[r] = forwarding
    
    return forwarding_tables

In [19]:
forwarding_tables=compute_forwarding_tables(routers)

For each router, this function:

1.Constructs a graph representation based on the router's LSDB

2.Runs Dijkstra's algorithm to find shortest paths to all destinations

3.Traces back the paths to determine the next hop for each destination

4.Compiles these next-hop decisions into a forwarding table

Printing the forwarding tables

In [20]:
def print_forwarding_table(router_id, forwarding_tables=forwarding_tables):
    """
    Prints the forwarding table for a specific router.
    
    Parameters:
    router_id (int): ID of the router whose forwarding table will be printed
    forwarding_tables (dict): Dictionary containing all router forwarding tables
    """
    if router_id not in forwarding_tables:
        print(f"Error: Router {router_id} not found in forwarding tables")
        return
    
    # Get this router's forwarding table
    router_table = forwarding_tables[router_id]
    
    # Print header
    print(f"\n{'='*50}")
    print(f"FORWARDING TABLE FOR ROUTER {router_id}")
    print(f"{'-'*50}")
    print(f"{'Destination':<15} | {'Next Hop':<15}")
    print(f"{'-'*15}-+-{'-'*15}")
    
    # Sort destinations for consistent display
    for dest in sorted(router_table.keys()):
        next_hop = router_table[dest]
        if next_hop is None:
            next_hop_str = "UNREACHABLE"
        else:
            next_hop_str = str(next_hop)
        print(f"{dest:<15} | {next_hop_str:<15}")
    
    print(f"{'='*50}")
    print(f"Total destinations: {len(router_table)}\n")


In [21]:
print_forwarding_table(20)


FORWARDING TABLE FOR ROUTER 20
--------------------------------------------------
Destination     | Next Hop       
----------------+----------------
1               | 1              
2               | 2              
3               | 3              
4               | 21             
5               | 21             
6               | 21             
7               | 7              
8               | 8              
9               | 9              
10              | 12             
11              | 11             
12              | 12             
13              | 12             
14              | 21             
15              | 15             
16              | 16             
17              | 21             
18              | 21             
19              | 21             
21              | 21             
22              | 22             
23              | 23             
24              | 12             
25              | 25             
26              | 21             

# (c) Function that takes source and destination IDs and provide the routing path

In [ ]:
def get_routing_path(src, dst, forwarding_tables=forwarding_tables):
    path = [src]
    current = src
    while current != dst:
        next_hop = forwarding_tables[current].get(dst)
        if next_hop is None:
            return None  # No path
        path.append(next_hop)
        current = next_hop
    return path

In [ ]:
get_routing_path(1,2)

[1, 27, 2]